#### The Goal of these notebooks is only to understand advanced data analytic tasks and walk through them using Scala. 

In [1]:
val name = "Shweta Purushe"

name = Shweta Purushe


Shweta Purushe

In [14]:
println(s"Hello ${name}")

Hello Shweta Purushe


In [1]:
print(sc.version) // Spark Version
print(util.Properties.versionString) // Scala version

2.3.1version 2.11.8

In [2]:
// Even though we give it a path no data is loaded until an action is performed on RDD
val rawblocks = sc.textFile("linkage") // RDD on the cluster 

rawblocks = linkage MapPartitionsRDD[1] at textFile at <console>:28


linkage MapPartitionsRDD[1] at textFile at <console>:28

In [11]:
rawblocks.first.getClass

class java.lang.String

### Reading data from the cluster (here stand alone Spark cluster) to client (Actions)

In [3]:
rawblocks.first

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"

In [3]:
// Lets try some more 
val head = rawblocks.take(10)// head is now available on our client (via the take action)
head.length

head = Array("id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match", 37291,53113,0.833333333333333,?,1,?,1,1,1,1,0,TRUE, 39086,47614,1,?,1,?,1,1,1,1,1,TRUE, 70031,70237,1,?,1,?,1,1,1,1,1,TRUE, 84795,97439,1,?,1,?,1,1,1,1,1,TRUE, 36950,42116,1,?,1,1,1,1,1,1,1,TRUE, 42413,48491,1,?,1,?,1,1,1,1,1,TRUE, 25965,64753,1,?,1,?,1,1,1,1,1,TRUE, 49451,90407,1,?,1,?,1,1,1,1,0,TRUE, 39932,40902,1,?,1,?,1,1,1,1,1,TRUE)


10

In [4]:
head.foreach(println)

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"
37291,53113,0.833333333333333,?,1,?,1,1,1,1,0,TRUE
39086,47614,1,?,1,?,1,1,1,1,1,TRUE
70031,70237,1,?,1,?,1,1,1,1,1,TRUE
84795,97439,1,?,1,?,1,1,1,1,1,TRUE
36950,42116,1,?,1,1,1,1,1,1,1,TRUE
42413,48491,1,?,1,?,1,1,1,1,1,TRUE
25965,64753,1,?,1,?,1,1,1,1,1,TRUE
49451,90407,1,?,1,?,1,1,1,1,0,TRUE
39932,40902,1,?,1,?,1,1,1,1,1,TRUE


In [5]:
// function to remove the records which contain id. 
def contains_id(line:String):Boolean = {line.contains("id_")}

contains_id: (line: String)Boolean


In [6]:
head.filter(contains_id).foreach(println)//our filter returns the headers and thus works

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"


In [7]:
head.filterNot(contains_id).length// the rest of the records which dont contain "id_" and are the ones we want

9

### Client to Cluster

In [14]:
// now running the above filter against the WHOLE rdd
val filtered_rdd = rawblocks.filter(x => !contains_id(x))

filtered_rdd = MapPartitionsRDD[2] at filter at <console>:32


MapPartitionsRDD[2] at filter at <console>:32

### Moving onto the DataFrame API

In [4]:
val df = spark.read.format("csv")
        .option("inferSchema", "true")
        .option("header", "true")
        .option("nullValue", "?")
        .load("linkage/*.csv")

df = [id_1: string, id_2: string ... 10 more fields]


[id_1: string, id_2: string ... 10 more fields]

In [5]:
df.first

[3148,8326,1,null,1,null,1,1,1,1,1,true]

In [6]:
df.printSchema()

root
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [22]:
df.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

In [7]:
df.groupBy("is_match").count().show()
// Another way to do it is register the df as a temp view and query it using the Sql API

+--------+-------+
|is_match|  count|
+--------+-------+
|    null|      1|
|    true|  20931|
|   false|5728201|
+--------+-------+



In [20]:
df.describe().select("summary", "id_1", "cmp_fname_c2", "cmp_plz").show()

+-------+--------------------+-------------------+-------------------+
|summary|                id_1|       cmp_fname_c2|            cmp_plz|
+-------+--------------------+-------------------+-------------------+
|  count|             5749133|             103699|            5736289|
|   mean|   33324.47979999771|  0.900008998936421|0.00552866147434343|
| stddev|   23659.86139888655|0.27133067681523776|0.07414914925420046|
|    min|0.000235404896421846|                  0|                  0|
|    max|                9999|                  1|                  1|
+-------+--------------------+-------------------+-------------------+



In [ ]:
//The above functionality will need several lines of code if we use the SparkSQL API
// import org.apache.spark.sql.functions._ 
// df.groupBy().agg(mean(), stf)

In [21]:
df.count()

5749133